In [2]:
# RUN THIS CELL: it loads some style files
from IPython.core.display import HTML, display, Math
with open( './style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

In [3]:
from scipy.stats import binom                               # lib. for statistical functions
from ipywidgets import interactive, FloatSlider, IntSlider  # lib. for interactive graphic
from bokeh.io import push_notebook, show, output_notebook   # lib. for graphic output
from bokeh.plotting import figure
output_notebook()
options = dict(plot_height=250,plot_width=700,
               tools="pan,wheel_zoom,reset,save,crosshair,box_select")
def printred(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#a02">' + s + '</pre>') )
def printgrn(s):
    display(HTML('<pre style="font: bold 12pt Courier, serif;color:#080">' + s + '</pre>') )

Loading BokehJS ...

# Prevalenza mancinismo tra gli studenti di STdM

Il $10\%$ è delle persone sono mancine. Vogliamo stabilire se i mancini hanno una preferenza per le STdM. Supponiamo di avere un campione di $100$ studenti e che possiamo ritenere un campione casuale.

Il test di ipotesi che faremo è

$H_0:\kern3.5ex p=0.1\quad$ la prevalenza del mancinismo tra i biotecnologi è identica

$H_A:\kern3.5ex p> 0.1\quad$  la prevalenza del mancinismo tra i biotecnologi è maggiore

Il test è una variabile aleatoria $X$ a valori in $\{0,\dots,100\}$. 

Condizionando a $H_0$ otteniamo $X\sim {\rm B}(100,\ 0.1)$. 

Condizionando a $H_A$ otteniamo $X\sim {\rm B}(100,\ p)$ con $p> 0.1$ ignota.

**Domanda**

Nel nostro campione abbiamo contato $16$ mancini.

Assumendo noti i valori della funzione $f(k)=\Pr(X\le k)$ per $X\sim {\rm B}(100,\ 0.1)$ si calcoli con che significatività possiamo rigettare l'ipotesi nulla dati i 16 mancini trovati? 

Ovvero progettiamo il test più restrittivo che è positivo per quello che abbiamo osservato: 16.

Calcoliamo la significatività in questo caso.

Significatività = $\Pr(X\ge 16 | H_0) = 1 - f(15)$

La significatività del test più restrittivo che rigetta l'ipotesi nulla si chiama <mark>p-valore</mark>.

L'espressione usata è: *rigettiamo l'ipotesi nulla con p-varore $1 - f(15)$*

In [4]:
# Calcolo numerico
1 - binom.cdf(15,100,0.1)

0.03989052711108354

Il valore numerico calcolato qui sopra è $<0.05$. Questo è giuticato sufficientemente piccolo nelle scienze mediche e sociali - nelle scenze fisico/chimiche ci sono altri standard.

## Visualizzazione grafica

Gli errori del I tipo per diverse regioni di rifiuto sono plottate nel grafico sottostante.

In [5]:
n = 100; p0 = 0.1; k_max = 16                                # Set the initial values of the parameters
x = range(0,n+1)

plot = figure(title="PMF of X ~ B(100, 0.1).",  # Create an empty figure
              x_axis_label = "# successes", y_axis_label = "probability", 
              x_range=(0,n*0.3), y_range=(0,max(binom.pmf(x,n,p0)*1.1)), **options )               
plot.title.text_font="courier"
plot.title.text_font_size="12pt"


r1 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='green', alpha=0.5) # Initialize barplot
r2 = plot.vbar(x, top=[0 for i in x], bottom=0, width=.9, color='red',   alpha=0.5) # Initialize barplot
        
def f(k, one_tail=True): 
    k1 = int(n*p0)-k; k2 = int(n*p0)+k
    data = {'x':x, 'top':  binom.pmf(x, n, p0) }
    r1.data_source.data = data
    if not one_tail :
        printred('T = [0, {}]  ∪  [ {}, 100 ] zona di rifiuto'.format(k1,k2) ) 
        printred('Pr( T | p = {:.2f}) = α = {:.3f} (falso positivo) ERRORE I tipo'.format(p0,binom.cdf(k1, n, p0)-binom.cdf(k2-1, n, p0) +1) ) 
        x_rif = [ i for i in x if i <= int(n*p0)-k or i>= int(n*p0) + k ]
    else: 
        printred('T = [ {}, 100 ] zona di rifiuto'.format(k2) )
        printred('Pr( T | p = {:.2f}) = α = {:.3f} (falso positivo) ERRORE I tipo'.format(p0, 1 - binom.cdf(k2-1, n, p0) ) )
        x_rif = [ i for i in x if i>= int(n*p0) + k ]
    data2 = {'x':x_rif, 'top':binom.pmf(x_rif, n, p0) }
    r2.data_source.data = data2
    push_notebook()

show(plot, notebook_handle=True)                                                    # Show barplot
w = interactive(f,
         #tail = ToggleButtons(description="tail(s) ", options=['1 tail', '2 tails']),
         k = IntSlider  (description=" ", min=1, max=k_max, step=1, value=k_max),)
display(w)

interactive(children=(IntSlider(value=16, description=' ', max=16, min=1), Checkbox(value=True, description='o…